# CSC 481 Project 

Ayan Patel, Robert Hensley

* [Gowalla Dataset](https://snap.stanford.edu/data/loc-gowalla.html)



---



In [5]:
# !pip install snap-stanford
!pip install python-igraph
!pip install geopy

In [2]:
# import snap ???
import igraph
from igraph import *

import pandas as pd

import time
from datetime import datetime

import geopy
from geopy.geocoders import Nominatim

import itertools

## Loading Data

* this can takes 3 - 5 minutes to fully load all dataframes

In [3]:
data_path = 'data/'

# creating dataframes
edges = pd.read_csv(data_path + 'Gowalla_edges.txt', sep='\t', names=['u1', 'u2'])
totalCheckins = pd.read_csv(data_path + 'Gowalla_totalCheckins.txt', sep='\t',
                            names=['user', 'time', 'lat', 'long', 'locid'])

totalCheckins['time'] = totalCheckins['time'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ"))

In [4]:
totalCheckins

,user,time,lat,long,locid
0,0,2010-10-19 23:55:27,30.235909,-97.795140,22847
1,0,2010-10-18 22:17:43,30.269103,-97.749395,420315
2,0,2010-10-17 23:42:03,30.255731,-97.763386,316637
3,0,2010-10-17 19:26:05,30.263418,-97.757597,16516
4,0,2010-10-16 18:50:42,30.274292,-97.740523,5535878
...,...,...,...,...,...
6442887,196578,2010-06-11 13:32:26,51.742988,-0.488065,906885
6442888,196578,2010-06-11 13:26:45,51.746492,-0.490780,965121
6442889,196578,2010-06-11 13:26:34,51.741916,-0.496729,1174322
6442890,196585,2010-10-08 21:01:49,50.105516,8.571525,471724


In [5]:
edges

,u1,u2
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
1900649,196586,196539
1900650,196587,196540
1900651,196588,196540
1900652,196589,196547


### User Notes

* userCheckins appears to cover only 1245 users
  * this makes me wonder if the userIDs match the edges userIDs (most likely not)
  * therefore it might be better to create our own graphs based off the checkin data and ignore the edges data
    * this means we cannot make the assumptions that two users are friends 
    * we should test to see if our predicted meetups (same location at around the same time) match the meetups in the edges file
  * also notice that user ID's appear to range from 0 - 1440, but there are only 1245 rows, implying some missing users


* we can get an idea of where the users are generally located by gather the mean of their latitude on longitudinal locations
* other user attributes to collect:
  * earliest time checked in
  * latest time checked in 
    * this shows the general timespan the user has been on the app 

In [9]:
totalCheckins['user'].max()

196585

In [10]:
users = totalCheckins.groupby('user').mean()[['lat', 'long']]
users

,lat,long
user,,
0,33.558308,-97.894601
1,47.204338,4.499703
2,35.659617,-120.016716
4,36.800202,-124.714027
5,32.290069,-96.123008
...,...,...
196544,-25.433409,-49.281533
196561,37.528650,-122.004623
196577,51.514905,-0.081277


* look at how many checkins the average user makes
  * this seems almost too big?
  * I assume this is a collection of users classified as very active on Gowalla

In [11]:
# average amounts of visits per user
totalCheckins.shape[0] / users.shape[0]

60.16221566503567

### Place Notes

* same grouping process as users
* for the scope of this project, we might want to only focus on places with like 5 or more visits

In [4]:
places = totalCheckins.groupby('locid').agg(['mean', 'count'])[['lat', 'long']]

places.columns = ['lat_mean', 'count_drop', 'long_mean', 'visits']
places = places.drop(columns = ['count_drop'])

In [5]:
places.sort_values('visits')

,lat_mean,long_mean,visits
locid,,,
5977757,13.447479,101.019928,1
951900,50.718459,-1.877455,1
951901,41.543670,-83.582070,1
3733285,50.084312,18.212821,1
3733273,33.621132,-112.185371,1
...,...,...,...
58725,59.650051,17.932262,3476
10259,32.897462,-97.040348,4083
9410,30.201557,-97.667127,4713


In [12]:
places_more_than_once = places[places.visits > 1]
places_more_than_once

,lat_mean,long_mean,visits
locid,,,
8904,39.052318,-94.607499,12
8932,32.927662,-97.254356,16
8936,39.053318,-94.591995,12
8938,39.052824,-94.590311,130
8947,37.331880,-122.029631,570
...,...,...,...
5912882,30.269072,-97.749409,3
5914456,58.449462,14.893234,2
5927439,41.229163,1.773204,2


#### Geolocation Data (geopy)

* our queries use the [Nominatim API](https://nominatim.org/release-docs/develop/api/Overview/) by default

Let's try to find the country all this data is from. We will query location information using [geopy](https://geopy.readthedocs.io/en/stable/). Below is a sample query.

In [14]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()

location = geolocator.reverse("39.052318, -94.607499")

C:\Users\Ayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
location.raw

{'place_id': 251278814,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 552493654,
 'lat': '39.052405',
 'lon': '-94.607436',
 'display_name': '4154, State Line Road, Country Club Plaza, Kansas City, Wyandotte County, Missouri, 66103, United States of America',
 'address': {'house_number': '4154',
  'road': 'State Line Road',
  'suburb': 'Country Club Plaza',
  'city': 'Kansas City',
  'county': 'Wyandotte County',
  'state': 'Missouri',
  'postcode': '66103',
  'country': 'United States of America',
  'country_code': 'us'},
 'boundingbox': ['39.052355', '39.052455', '-94.607486', '-94.607386']}

In [16]:
location.raw['address']['country']

'United States of America'

This gives the following important data:
* name 'address > tourism'
* city 'address > city'
* country 'address > country'

However this data lacks:
* context of the location (is it a pub, park, etc.)
    * we can get this information by querying OpenStreetMap using it's place_id, 

Given the limited time for this project, I feel like city and country information should be informative enough.

**Naming Conventions**

* subregions vary between different countries 
    * i.e. states and counties are not found in Germany, boroughs are found in Germany but not the US
    * this means the easiest thing to analyze right now are places < cities < countries
    * this problem can be resolved by focusing on one Country at a time
* the above problem unfortunately also occurs for city names
    * citities can also be called "towns, villages, municipalities, etc.)

In [20]:
location.raw

place = {"osm_place_id"}

{'place_id': 251278814,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 552493654,
 'lat': '39.052405',
 'lon': '-94.607436',
 'display_name': '4154, State Line Road, Country Club Plaza, Kansas City, Wyandotte County, Missouri, 66103, United States of America',
 'address': {'house_number': '4154',
  'road': 'State Line Road',
  'suburb': 'Country Club Plaza',
  'city': 'Kansas City',
  'county': 'Wyandotte County',
  'state': 'Missouri',
  'postcode': '66103',
  'country': 'United States of America',
  'country_code': 'us'},
 'boundingbox': ['39.052355', '39.052455', '-94.607486', '-94.607386']}

In [5]:
places

,lat_mean,long_mean,visits
locid,,,
8904,39.052318,-94.607499,12
8932,32.927662,-97.254356,16
8936,39.053318,-94.591995,12
8938,39.052824,-94.590311,130
8947,37.331880,-122.029631,570
...,...,...,...
5975123,38.020788,-7.874773,1
5976149,12.939186,100.882264,1
5976173,13.668828,100.644486,1


In [103]:
location = geolocator.reverse("39.052318, -94.607499")

In [105]:
location.raw

{'place_id': 251278814,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 552493654,
 'lat': '39.052405',
 'lon': '-94.607436',
 'display_name': '4154, State Line Road, Country Club Plaza, Kansas City, Wyandotte County, Missouri, 66103, United States of America',
 'address': {'house_number': '4154',
  'road': 'State Line Road',
  'suburb': 'Country Club Plaza',
  'city': 'Kansas City',
  'county': 'Wyandotte County',
  'state': 'Missouri',
  'postcode': '66103',
  'country': 'United States of America',
  'country_code': 'us'},
 'boundingbox': ['39.052355', '39.052455', '-94.607486', '-94.607386']}

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()

sorted_places = places.sort_values('visits', ascending=False).reset_index()

In [6]:
sorted_places = places.sort_values('visits', ascending=False).reset_index()
sorted_places

,locid,lat_mean,long_mean,visits
0,55033,59.330158,18.058079,5811
1,19542,37.616356,-122.386150,5662
2,9410,30.201557,-97.667127,4713
3,10259,32.897462,-97.040348,4083
4,58725,59.650051,17.932262,3476
...,...,...,...,...
1280964,1071645,8.038478,98.815826,1
1280965,1071647,42.553000,-83.175745,1
1280966,1071650,34.614540,-92.498446,1
1280967,1071652,29.756889,-95.365663,1


In [7]:
places_top10 = sorted_places[:10000]

places_top10

,locid,lat_mean,long_mean,visits
0,55033,59.330158,18.058079,5811
1,19542,37.616356,-122.386150,5662
2,9410,30.201557,-97.667127,4713
3,10259,32.897462,-97.040348,4083
4,58725,59.650051,17.932262,3476
...,...,...,...,...
9995,230547,36.005830,-115.084607,56
9996,45183,35.655814,-97.470548,56
9997,164356,48.101030,11.645453,56
9998,234171,63.825514,20.261439,56


In [8]:
places_top5 = sorted_places[:5000]
places_top5

,locid,lat_mean,long_mean,visits
0,55033,59.330158,18.058079,5811
1,19542,37.616356,-122.386150,5662
2,9410,30.201557,-97.667127,4713
3,10259,32.897462,-97.040348,4083
4,58725,59.650051,17.932262,3476
...,...,...,...,...
4995,326434,36.114043,-115.174356,84
4996,16355,32.783310,-96.805274,84
4997,35604,34.064908,-83.984149,84
4998,129858,14.583646,121.057191,84


In [9]:
top10_checkins = totalCheckins[totalCheckins.locid.isin(places_top10['locid'])]
top10_checkins

,user,time,lat,long,locid
0,0,2010-10-19 23:55:27,30.235909,-97.795140,22847
1,0,2010-10-18 22:17:43,30.269103,-97.749395,420315
2,0,2010-10-17 23:42:03,30.255731,-97.763386,316637
3,0,2010-10-17 19:26:05,30.263418,-97.757597,16516
4,0,2010-10-16 18:50:42,30.274292,-97.740523,5535878
...,...,...,...,...,...
6442635,196528,2010-10-11 01:42:02,13.766876,100.570672,761001
6442643,196528,2010-10-04 11:55:05,13.766876,100.570672,761001
6442650,196528,2010-09-30 02:33:25,13.689897,100.748320,23519
6442698,196541,2010-09-24 11:26:52,51.496105,-0.171834,23876


In [10]:
top10_edges = edges[edges.u1.isin(top10_checkins['user'])]
top10_edges = top10_edges[top10_edges.u2.isin(top10_checkins['user'])]
top10_edges

,u1,u2
1,0,2
3,0,4
4,0,5
6,0,7
7,0,8
...,...,...
1900428,196460,195795
1900446,196472,195869
1900474,196489,196024
1900485,196494,196033


In [11]:
top5_checkins = totalCheckins[totalCheckins.locid.isin(places_top5['locid'])]
top5_checkins

,user,time,lat,long,locid
1,0,2010-10-18 22:17:43,30.269103,-97.749395,420315
2,0,2010-10-17 23:42:03,30.255731,-97.763386,316637
3,0,2010-10-17 19:26:05,30.263418,-97.757597,16516
4,0,2010-10-16 18:50:42,30.274292,-97.740523,5535878
5,0,2010-10-12 23:58:03,30.261599,-97.758581,15372
...,...,...,...,...,...
6442556,196508,2010-09-04 12:16:05,51.507300,-0.221014,23886
6442621,196524,2010-09-29 22:57:13,40.763609,-73.972943,23741
6442622,196524,2010-09-29 22:56:47,40.763861,-73.972932,12535
6442650,196528,2010-09-30 02:33:25,13.689897,100.748320,23519


In [12]:
top5_edges = edges[edges.u1.isin(top5_checkins['user'])]
top5_edges = top5_edges[top5_edges.u2.isin(top5_checkins['user'])]
top5_edges

,u1,u2
1,0,2
3,0,4
4,0,5
6,0,7
7,0,8
...,...,...
1900407,196445,195692
1900428,196460,195795
1900446,196472,195869
1900474,196489,196024


In [ ]:
import json

location_json = {}
location_json['places'] = []

# choose place to start writing files
i = 1000
chunk = 1000
places_temp = sorted_places[i:]

for index, row in places_temp.iterrows():
    
    location = geolocator.reverse(str(row["lat_mean"]) + ", " + str(row["long_mean"])).raw
    
    location_json['places'].append({'locid' : row['locid'], 'osm_json' : location})
    
    if index % chunk == 0 and index != i:
        
        with open('data/json/' + str(index) + '.json', 'w') as outfile:
            json.dump(location_json, outfile)
        
        print(str(index) + ' index JSON written')
        
        location_json['places'] = []

1000 index JSON written


In [ ]:
    city = ''
    
    # cities
    try:
        city = location['address']['city']
    except:
        pass
    
    try:
        city = location['address']['town']
    except:
        pass
    
    try:
        city = location['address']['village']
    except:
        pass
    
    # skip entities with no city information
    if city == "":
        print("no city")
        print(location)
        pass
  
    location_append.append({
        'osm_place_id' : location['place_id'],
        'city' : city,
        'country' : location['address']['country'],
        'country_code' : location['address']['country_code']
    })

## Graph Representations

* having some issues importing Stanford Snap (conflicts with another library called snap)
* I'm going to try iGraph in the meantime

### Limitations

* the max userID will be the number of edges required in the graph (assumption)
* using the maximum number of nodes in the graph (196591) takes a long time to generate the graph
  * this I would assume makes the algorithms for these graphs tricky to run
  * I could also be adding the edges in an unefficient way (explore ways of generating graphs)
    * might be better to add edges in batches instead of one edge at a time (explore this)

#### Ways to set limits
* we can limit the groups to the first n number of users (example only nodes 0 - 1000)
  * however under the assumption that userIDs are added based on the time the user adds themselves to the network, then there will be a lot of graphs that aren't connected because they are in geographically seperate locations
* a better soltuion would be to take a group of n people in close proximety of one another
  * we could find the average latitude and longitude of a users and record this as the area they are generally in
  * we'll then randomly select a user and select the n amount of people closest to this user (ranking by the distance between their average locations)


In [6]:
edgesG = Graph()
total_edges = max(edges['u1'].max(), edges['u2'].max())
total_edges

196590

In [18]:
edgesG.add_vertices(total_edges)

In [0]:
# this takes too long; there's got to be a better way!

for i in range(len(edges)):
  edgesG.add_edge(edges.loc[i, "u1"], edges.loc[i, "u2"])

**Creating graph of user friendship**

In [6]:
#Faster way to make graph from df tuples
eG = igraph.Graph.TupleList(edges.itertuples(index=False), directed=False, weights=False)
eG.summary()

'IGRAPH UN-- 196591 1900654 -- \n+ attr: name (v)'

Finding the indegree of each user and identifying the user with maximum degree

In [7]:
#Finds degrees of each vertex and the max degree and its vertex - popularity
eG_degrees = eG.indegree()
print(max(eG_degrees))
eG_max_degree_index = eG_degrees.index(max(eG_degrees))
eG.vs[eG_max_degree_index]

29460


igraph.Vertex(<igraph.Graph object at 0x000001B08ABCAE58>, 307, {'name': 307})

Finding all the neighbors of the user with maximum degree, in other words all of the popular user's friends

In [8]:
eG.neighbors(eG_max_degree_index)

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 5,
 5,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 14,
 14,
 17,
 17,
 18,
 18,
 20,
 20,
 21,
 21,
 22,
 22,
 24,
 24,
 30,
 30,
 35,
 35,
 37,
 37,
 39,
 39,
 43,
 43,
 44,
 44,
 46,
 46,
 49,
 49,
 50,
 50,
 53,
 53,
 54,
 54,
 55,
 55,
 60,
 60,
 61,
 61,
 65,
 65,
 67,
 67,
 68,
 68,
 69,
 69,
 71,
 71,
 72,
 72,
 73,
 73,
 75,
 75,
 80,
 80,
 82,
 82,
 90,
 90,
 91,
 91,
 94,
 94,
 96,
 96,
 112,
 112,
 113,
 113,
 114,
 114,
 115,
 115,
 117,
 117,
 119,
 119,
 123,
 123,
 140,
 140,
 141,
 141,
 143,
 143,
 145,
 145,
 153,
 153,
 158,
 158,
 168,
 168,
 174,
 174,
 178,
 178,
 180,
 180,
 182,
 182,
 184,
 184,
 188,
 188,
 194,
 194,
 198,
 198,
 202,
 202,
 205,
 205,
 207,
 207,
 209,
 209,
 210,
 210,
 211,
 211,
 215,
 215,
 216,
 216,
 218,
 218,
 219,
 219,
 220,
 220,
 223,
 223,
 228,
 228,
 236,
 236,
 237,
 237,
 241,
 241,
 242,
 242,
 243,
 243,
 244,
 244,
 245,
 245,
 248,
 248,
 261,
 261,
 266,
 266,
 269,
 269,
 279,
 279,


**Creating graph of users that visited the top 10k places**

In [11]:
top10_eG = igraph.Graph.TupleList(top10_edges.itertuples(index=False), directed=False, weights=False)
top10_eG.summary()

'IGRAPH UN-- 63954 633948 -- \n+ attr: name (v)'

Identifying the user with max degree, most popular 

In [10]:
top10_eG_degrees = top10_eG.indegree()
print(max(top10_eG_degrees))
top10_eG_max_degree_index = top10_eG_degrees.index(max(top10_eG_degrees))
top10_eG.vs[top10_eG_max_degree_index]

14630


igraph.Vertex(<igraph.Graph object at 0x000001CCA9533E58>, 268, {'name': 307})

Using pagerank to find the most popular user - note it matches above method

In [24]:
top10_eG_PR = top10_eG.pagerank()
print(max(top10_eG_PR))
print(top10_eG_PR.index(max(top10_eG_PR)))
top10_eG_PR

0.01109515802806296
268


[0.0005601428985739857,
 0.0012451564315959104,
 3.9544888079393464e-05,
 0.0005378165257966492,
 4.1839343459879194e-05,
 4.509137461765956e-05,
 3.139457580824911e-05,
 2.2394456969758714e-05,
 1.8308983076667677e-05,
 3.7075088254714646e-05,
 4.2564765231561685e-05,
 0.00012220473017581946,
 3.504858739399901e-05,
 3.077570251426852e-05,
 0.00011980908394106588,
 3.0447555293858315e-05,
 0.00014091226344829672,
 0.00025192650719313024,
 0.00021114524289245172,
 1.1705708578519792e-05,
 1.585730333712413e-05,
 2.4330927487731063e-05,
 4.911319873694109e-05,
 6.216401646213416e-05,
 1.986162431136007e-05,
 5.836984111221423e-05,
 3.6793734815685755e-05,
 0.00011446439056066688,
 3.3207052781472134e-05,
 8.262801306450721e-05,
 4.069785166706043e-05,
 2.4867214998666827e-05,
 2.456672176997244e-05,
 0.0002279537320801742,
 0.00018039844014468582,
 0.00010946901292076022,
 4.259099614362942e-05,
 0.00024576391209893444,
 8.500463345363405e-06,
 2.593635760059841e-05,
 2.1832071695049115

Finding all connected components, or clusters, of the graph, and the largest cluster

In [22]:
top10_eG_clusters = top10_eG.components()
print("Num of Clusters:", len(top10_eG_clusters))
top10_eG_giant_cluster = top10_eG_clusters.giant()
print("Num of Vertices in Largest cluster:", top10_eG_giant_cluster.vcount())

Num of Clusters: 683
Num of Vertices in Largest cluster: 62302


In [13]:
top10_eG.degree_distribution()

In [14]:
top10_eG.omega() #size of largest clique

20

In [13]:
top5_eG = igraph.Graph.TupleList(top5_edges.itertuples(index=False), directed=False, weights=False)
top5_eG.summary()

'IGRAPH UN-- 57260 570846 -- \n+ attr: name (v)'

In [14]:
top5_eG_edges = [edge.tuple for edge in top5_eG.es]

**Creating a graph between users and locations they have visited**

In [26]:
user_locids = totalCheckins[['user', 'locid']]
user_locids['locid'] = user_locids['locid'].apply(lambda x: 'L' + str(x))
user_locids

C:\Users\Ayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user,locid
0,0,L22847
1,0,L420315
2,0,L316637
3,0,L16516
4,0,L5535878
...,...,...
6442887,196578,L906885
6442888,196578,L965121
6442889,196578,L1174322
6442890,196585,L471724


In [27]:
user_loc_G = igraph.Graph.TupleList(user_locids.itertuples(index=False), directed=False, weights=False)
user_loc_G.summary()

'IGRAPH UN-- 1388061 6442892 -- \n+ attr: name (v)'

**Creating df of all users that visited top10 locations**

In [41]:
user_locid = totalCheckins[['user', 'locid']]
top10_user_locid = user_locid[user_locid.locid.isin(places_top10['locid'])]

In [42]:
top10_user_locid_groups = top10_user_locid.groupby('locid')['user'].apply(list).reset_index(name='users')
top10_user_locid_groups

,locid,users
0,8938,"[0, 65, 71, 107, 107, 107, 228, 256, 256, 256,..."
1,8947,"[24, 99, 119, 124, 130, 130, 130, 130, 130, 13..."
2,8956,"[138, 138, 138, 138, 138, 350, 486, 559, 560, ..."
3,8961,"[350, 350, 560, 1759, 2274, 2274, 2274, 2274, ..."
4,8964,"[0, 71, 75, 107, 107, 107, 107, 107, 107, 107,..."
...,...,...
9995,4997564,"[32, 41, 212, 225, 242, 252, 267, 267, 342, 35..."
9996,4998006,"[41, 97, 112, 212, 225, 242, 350, 401, 438, 47..."
9997,4998370,"[41, 97, 180, 212, 225, 225, 242, 350, 401, 47..."
9998,5341244,"[557, 3567, 4011, 4011, 4580, 6609, 7492, 7532..."


In [ ]:
#takes time to run but gets all combinations of users in each list that we can make edges for visitng same location
top10_user_locid_combo = top10_user_locid_groups['users'].apply(lambda x: list(itertools.combinations(x, 2))).reset_index(name='user_combos')
top10_user_locid_combo

**Grouping checkins by time and location**

In [38]:
time_loc = totalCheckins[['time', 'user', 'locid']]
time_loc = time_loc[time_loc.locid.isin(places_top10['locid'])]
time_loc['time'] = time_loc['time'].dt.floor('30T') #rounding timestamp to nearest 30 minutes
time_loc.groupby(['locid', 'time']).agg(['count'])

user
                            count
locid   time                     
8938    2009-08-29 02:00:00     1
        2009-09-26 13:00:00     1
        2009-09-27 01:30:00     1
        2009-09-27 23:00:00     1
        2009-10-06 22:00:00     1
...                           ...
5535878 2010-10-17 19:30:00     2
        2010-10-17 20:30:00     3
        2010-10-17 21:00:00     2
        2010-10-17 21:30:00     1
        2010-10-17 22:00:00     1

[1143358 rows x 1 columns]

The following shows a list of users that where at a location at the same time

In [19]:
time_loc5 = totalCheckins[['time', 'user', 'locid']]
time_loc5 = time_loc5[time_loc5.locid.isin(places_top5['locid'])]
time_loc5['time'] = time_loc5['time'].dt.floor('60T') #rounding timestamp to nearest hour
time_loc5_groups = time_loc5.groupby(['locid', 'time'])['user'].apply(list).reset_index(name='users')
time_loc5_groups = time_loc5_groups[time_loc5_groups['users'].apply(lambda x: len(x) > 2)]
time_loc5_groups

,locid,time,users
161,8947,2009-11-17 17:00:00,"[285, 487, 518]"
195,8947,2009-12-16 21:00:00,"[494, 494, 23349]"
229,8947,2010-01-27 17:00:00,"[130, 28602, 125329]"
283,8947,2010-03-06 00:00:00,"[6927, 125329, 127250]"
349,8947,2010-04-22 18:00:00,"[519, 5461, 11637]"
...,...,...,...
788693,5535878,2010-10-17 17:00:00,"[33053, 33711, 70217, 84630, 155582]"
788694,5535878,2010-10-17 18:00:00,"[716, 4726, 6701, 21757, 50446]"
788695,5535878,2010-10-17 19:00:00,"[5711, 10803, 103076]"
788696,5535878,2010-10-17 20:00:00,"[3155, 4229, 11238]"


In [20]:
time_loc5_groups_combo = time_loc5_groups['users'].apply(lambda x: list(itertools.combinations(x, 2))).reset_index(name='user_combos')
time_loc5_groups_combo

,index,user_combos
0,161,"[(285, 487), (285, 518), (487, 518)]"
1,195,"[(494, 494), (494, 23349), (494, 23349)]"
2,229,"[(130, 28602), (130, 125329), (28602, 125329)]"
3,283,"[(6927, 125329), (6927, 127250), (125329, 1272..."
4,349,"[(519, 5461), (519, 11637), (5461, 11637)]"
...,...,...
18502,788693,"[(33053, 33711), (33053, 70217), (33053, 84630..."
18503,788694,"[(716, 4726), (716, 6701), (716, 21757), (716,..."
18504,788695,"[(5711, 10803), (5711, 103076), (10803, 103076)]"
18505,788696,"[(3155, 4229), (3155, 11238), (4229, 11238)]"


In [24]:
time_loc5_groups_combo[:10]['user_combos'].apply(lambda x: sum(el in top5_eG_edges for el in x)).reset_index(name='friends')

,index,friends
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,0
7,7,0
8,8,0
9,9,0


Finding the popular time for people to visit a particular location

In [32]:
time_loc['time'] = time_loc['time'].dt.time
time_loc.groupby(['locid', 'time']).agg(['count'])

user
                 count
locid   time          
8938    00:00:00     2
        00:30:00     4
        01:00:00     2
        01:30:00     2
        02:00:00     1
...                ...
5535878 21:00:00     2
        21:30:00     1
        22:00:00     2
        23:00:00     1
        23:30:00     1

[264061 rows x 1 columns]

The following shows the most common time for people to visit that location as well as what user visited that location the most

In [34]:
time_loc.groupby(['locid']).agg(lambda x:x.value_counts().index[0])

,time,user
locid,,
8938,18:00:00,488
8947,19:30:00,518
8956,19:30:00,138
8961,17:00:00,4678
8964,13:00:00,406
...,...,...
4997564,18:00:00,25344
4998006,00:30:00,956
4998370,19:00:00,16356


## Map APIs

* important for querying specific information about locations people visit
* data visualization is not an essential part of the project, but is worth exploring if data visualization is easy

### Notes About OSM
* [OpenStreetMap Article](https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0)

* We will be using the Overpass API to get information about our locations
  * [Overpass Querying Language](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL)
  * [Site for Testing Overpass Queries](http://overpass-turbo.eu/)


* **node**: a specific lat, long point
* **relation**: a region (typically for buildings that require multiple polygons
  * probably will be using this for our project (need a way to query if a user's location is in a certain relation)
  * includes tags which can helps us 

In [0]:
# example of an overpass query that collects all the biergartens in germany
# there are a lot of biergartens in Germany lol; so the query will take long to run 
# notice that it includes every OpenStreetMap datatype (node, way, rel); maybe there are biergarten streets? (way)

import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [0]:
# notice how descriptive the API is for relations

data['elements'][5:10]

[{'id': 27318009,
  'lat': 52.4200885,
  'lon': 13.1763456,
  'tags': {'addr:city': 'Berlin',
   'addr:housenumber': '260',
   'addr:postcode': '14109',
   'addr:street': 'Kronprinzessinnenweg',
   'amenity': 'biergarten',
   'contact:email': 'info@loretta-berlin.de',
   'contact:fax': '+49 30 80105334',
   'contact:phone': '+49 30 80105333',
   'contact:website': 'https://www.loretta-wannsee.de/biergarten/',
   'name': 'Loretta',
   'opening_hours': '12:00+',
   'wheelchair': 'limited'},
  'type': 'node'},
 {'id': 27352197,
  'lat': 50.3638373,
  'lon': 7.5769021,
  'tags': {'amenity': 'biergarten', 'wheelchair': 'limited'},
  'type': 'node'},
 {'id': 27787909,
  'lat': 52.9822191,
  'lon': 8.845254,
  'tags': {'amenity': 'biergarten',
   'name': 'Schwarzbiergarten',
   'wheelchair': 'limited'},
  'type': 'node'},
 {'id': 29812167,
  'lat': 49.4814353,
  'lon': 10.993033,
  'tags': {'amenity': 'biergarten',
   'toilets:wheelchair': 'no',
   'wheelchair': 'limited'},
  'type': 'node'},

# Sources

A collection of sources to include in the final paper:

* [OpenStreetMap Article](https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0)